# Multiple treatments examples
- In this notebook, we want to demonstrate common use cases where we have multiple treatments (ab testing is not covered, as you can just use another notebook for it)
- Simple CausalTune training with multiple treatments
- Applying custom propensities and train CausalTune (for example, in Uplift models)
- ERUPT for multiple treatments

### Setting up the data and causal model: CausalityDataset
- This notebook demonstrates how to train CausalTune with multiple custom propensities.
- The easiest example here is uplift modeling. Suppose we trained a model, sent messages, launched an experiment.
- And now we want to train more complex model using previous round model as propensities.

In [35]:
%load_ext autoreload
%autoreload 2
import os, sys
import warnings
warnings.filterwarnings('ignore') # suppress sklearn deprecation warnings for now..

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from typing import List, Union
import random

# the below checks for whether we run dowhy, causaltune, and FLAML from source
root_path = root_path = os.path.realpath('../..')
try:
    import causaltune
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "causaltune"))

try:
    import dowhy
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "dowhy"))

try:
    import flaml
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "FLAML"))
    
    
    
from causaltune import CausalTune
from causaltune.datasets import synth_ihdp
from causaltune.data_utils import CausalityDataset
from causaltune.erupt import DummyPropensity, ERUPT
from causaltune.models.passthrough import passthrough_model

### Data and generating treatments
- We first illustrate the model setup with a subset of data from the Infant Health and Development Program (IHDP).
- Then we synthetically create multiple treatments

In [2]:
data = synth_ihdp(return_df=True).iloc[:,:10]

In [3]:
data.head()

,treatment,y_factual,x1,x2,x3,x4,x5,x6,x7,x8
0,1,5.599916,-0.528603,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0
1,0,6.875856,-1.736945,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0
2,0,2.996273,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0
3,0,1.366206,0.390083,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0
4,0,1.963538,-1.045229,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0


In [4]:
len(data)

- Here we just randomly create multiple treatments

In [5]:
np.random.seed(42)
def generate_treatments(N: int, n_groups: int) -> List[List[Union[int, float]]]:
    treatments = [random.randint(0, n_groups) for _ in range(N)]
    values = [[random.random() for _ in range(N)] for _ in range(n_groups+1)]
    row_sums = [sum(row) for row in values]
    probabilities = [[prob / row_sum for prob in row] for row, row_sum in zip(values, row_sums)]
    for i in range(N):
        sum_probs = sum(row[i] for row in probabilities)
        for row in probabilities:
            row[i] /= sum_probs

    return [treatments] + probabilities

N = 747
n_groups = 4

treatments = generate_treatments(N, n_groups)

In [6]:
data['treatment'] = treatments[0]
data['propensity_control'] = treatments[1]
data['propensity_treatment1'] = treatments[2]
data['propensity_treatment2'] = treatments[3]
data['propensity_treatment3'] = treatments[4]
data['propensity_treatment4'] = treatments[5]

data['propensity_selected_group'] = data.apply(
    lambda row: row['propensity_control'] if row['treatment'] == 0 else 
                                         (row['propensity_treatment1'] if row['treatment'] == 1 else 
                                         (row['propensity_treatment2'] if row['treatment'] == 2 else 
                                         (row['propensity_treatment3'] if row['treatment'] == 3 else
                                         (row['propensity_treatment4'] if row['treatment'] == 4 else None)))),
    axis=1
)

- Now we have `treatment` (0, 1, 2, 3, 4)
- Outcome `y_factual`
- Features `x1, x2, x3, x4, x5, x6, x7, x8`
- Propensities `control, treatment1, treatment2, treatment3, treatment4`
- selected treatment related propensity `propensity_selected_group`

In [7]:
data.head()

,treatment,y_factual,x1,x2,x3,x4,x5,x6,x7,x8,propensity_control,propensity_treatment1,propensity_treatment2,propensity_treatment3,propensity_treatment4,propensity_selected_group
0,2,5.599916,-0.528603,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,0.107949,0.341507,0.029807,0.257914,0.262823,0.029807
1,3,6.875856,-1.736945,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,0.329860,0.003811,0.270932,0.263306,0.132091,0.263306
2,1,2.996273,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,0.295283,0.082931,0.219673,0.200007,0.202105,0.082931
3,4,1.366206,0.390083,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,0.343119,0.191204,0.041045,0.227161,0.197471,0.197471
4,4,1.963538,-1.045229,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,0.269994,0.057299,0.402925,0.036241,0.233541,0.233541


Generally, at least four arguments have to be supplied to `CausalityDataset` if you want to train a model:
- `data`: input dataframe
- `treatment`: name of treatment column
- `outcomes`: list of names of outcome columns; provide as list even if there's just one outcome of interest
- `effect_modifiers`: list of names of feature columns;

In addition, if the propensities to treat are known, then provide the corresponding column name(s) via `propensity_modifiers`.

In [8]:
features = [
    'x1',
    'x2',
    'x3',
    'x4',
    'x5',
    'x6',
    'x7',
    'x8'
]

propensities = [
    'propensity_control',
    'propensity_treatment1',
    'propensity_treatment2',
    'propensity_treatment3',
    'propensity_treatment4'
]

In [9]:
cd = CausalityDataset(
    data=data,
    treatment='treatment',
    outcomes=['y_factual'],
    effect_modifiers = features,
    propensity_modifiers = propensities
)

- To transform categorical columns and do some manipulations

In [10]:
cd.preprocess_dataset()

### Train CausalTune with simple energy distance (multiple treatments)
- Here we fit a (selection of) model(s) to the data and score them with the energy distance metric

In [11]:
# training configs

# set evaluation metric
metric = "energy_distance"

# it's best to specify either time_budget or components_time_budget, 
# and let the other one be inferred; time in seconds
components_time_budget = 10

# specify training set size
train_size = 0.7

Now if `outcome_model="auto"` in the CausalTune constructor, we search over a simultaneous search space for the EconML estimators and for FLAML wrappers for common regressors. The old behavior is now achieved by `outcome_model="nested"` (Refitting AutoML for each estimator).

You can also preprocess the data in the CausalityDataset using one of the popular category encoders: OneHot, WoE, Label, Target.

In [12]:
ct = CausalTune(
    estimator_list=[
        "DomainAdaptationLearner",
        "CausalForestDML",
        "ForestDRLearner",
    ],
    metric=metric,
    verbose=1,
    components_time_budget=components_time_budget,
    train_size=train_size,
    outcome_model="auto",
)

In [36]:
# run causaltune
ct.fit(data=cd, outcome=cd.outcomes[0])

print('---------------------')
# return best estimator
print(f"Best estimator: {ct.best_estimator}")
# config of best estimator:
print(f"Best config: {ct.best_config}")
# best score:
print(f"Best score: {ct.best_score}")

In [14]:
preds = ct.effect(cd.data)

In [15]:
preds

array([[-0.17528909, -0.10834053, -0.27350324, -0.27354165],
       [ 0.19621234,  0.30247269, -0.1484149 , -0.25184741],
       [ 0.11457749,  0.02436523, -0.04183348,  0.23708047],
       ...,
       [-0.96454594, -0.57960515,  0.08710834, -0.38235969],
       [-0.69400728, -0.03214504, -0.0508105 ,  0.27500816],
       [-0.02277706, -0.39042558,  0.05169252, -0.22645511]])

- Here we want to get results from the predictions
- We do naive argmax, but it is also recommended to use Thompson Sampling

In [16]:
predicted_treatments = np.argmax(preds, axis=1) + 1

### Train CausalTune with custom multiple propensities
In some settings such as uplift modelling, the experiment / study is based on heterogeneous treatment propensities known to the researcher / experimenter. An array of treatment propensities can be directly supplied to CausalTune in the data instantiation of the `CausalityDataset`. This can, e.g. be done by 
```
cd = CausalityDataset(
    ...
    propensity_modifiers=[<individual_treatment_propensity_column_name>]
    ...
)
```
and then using the `passthrough_model` as follows

In [17]:
propensity_model=passthrough_model(
    cd.propensity_modifiers, include_control=True
)

In [18]:
# set evaluation metric, if we want to use custom propensities, we should use "psw_energy_distance" metric
metric = "psw_energy_distance"

Now if `outcome_model="auto"` in the CausalTune constructor, we search over a simultaneous search space for the EconML estimators and for FLAML wrappers for common regressors. The old behavior is now achieved by `outcome_model="nested"` (Refitting AutoML for each estimator).

You can also preprocess the data in the CausalityDataset using one of the popular category encoders: OneHot, WoE, Label, Target.

In [19]:
ct2 = CausalTune(
    estimator_list=[
        "DomainAdaptationLearner",
        "CausalForestDML",
        "ForestDRLearner",
    ],
    metric=metric,
    verbose=1,
    components_time_budget=components_time_budget,
    train_size=train_size,
    propensity_model=propensity_model,
    outcome_model="auto"
)

In [37]:
# run causaltune
ct2.fit(data=cd, outcome=cd.outcomes[0])

print('---------------------')
# return best estimator
print(f"Best estimator: {ct2.best_estimator}")
# config of best estimator:
print(f"Best config: {ct2.best_config}")
# best score:
print(f"Best score: {ct2.best_score}")

In [21]:
preds2 = ct2.effect(cd.data)

In [22]:
preds2

array([[-0.82748584, -0.38301612, -0.28167136,  0.59362769],
       [-0.47553063, -1.32764429, -0.78241589, -0.11771117],
       [-0.56505201, -2.38952635, -1.25667425, -1.26012244],
       ...,
       [ 0.35639461,  0.81618186, -0.33151397, -0.45705827],
       [ 0.23314607,  0.54855842,  0.42367241,  2.13179076],
       [ 0.52856153,  0.5723138 ,  0.21125856,  0.57083639]])

- Here we want to get results from the predictions
- We do naive argmax, but it is also recommended to use Thompson Sampling

In [23]:
predicted_treatments2 = np.argmax(preds2, axis=1) + 1

### Run ERUPT with multiple propensities
Below we demonstrate how to use Estimated Response Under Proposed Treatment (ERUPT) to estimate the average treatment effect had the treatment been assigned randomly. Recall that the dataset used in this example is constructed in a way that the treatment propensity is a function of a unit's covariates.

In [24]:
np.random.seed(42)
test = synth_ihdp(return_df=True).iloc[:,:10]
N = 747
n_groups = 4

treatments = generate_treatments(N, n_groups)

test['treatment'] = treatments[0]
test['propensity_control'] = treatments[1]
test['propensity_treatment1'] = treatments[2]
test['propensity_treatment2'] = treatments[3]
test['propensity_treatment3'] = treatments[4]
test['propensity_treatment4'] = treatments[5]

test['propensity_selected_group'] = data.apply(
    lambda row: row['propensity_control'] if row['treatment'] == 0 else 
                                         (row['propensity_treatment1'] if row['treatment'] == 1 else 
                                         (row['propensity_treatment2'] if row['treatment'] == 2 else 
                                         (row['propensity_treatment3'] if row['treatment'] == 3 else
                                         (row['propensity_treatment4'] if row['treatment'] == 4 else None)))),
    axis=1
)

def randomize(value):
    return value + np.random.uniform(-1, 1)

for col in features:
    test[col] = test[col].apply(randomize)

test['y_factual'] = test['y_factual'].apply(randomize)
test['treatment'] = test['treatment'].sample(len(test))

In [25]:
test = test[test['treatment'] != 0].reset_index(drop=True)

In [26]:
test.head()

,treatment,y_factual,x1,x2,x3,x4,x5,x6,x7,x8,propensity_control,propensity_treatment1,propensity_treatment2,propensity_treatment3,propensity_treatment4,propensity_selected_group
0,4,6.175675,-0.835516,-0.989058,0.720254,2.409933,0.096092,1.619043,0.140574,0.329478,0.143508,0.009668,0.249150,0.340022,0.257652,0.263306
1,2,2.253838,-1.733191,0.276954,-0.228963,0.204594,0.789086,-0.205158,0.050398,-0.178537,0.219851,0.395494,0.271965,0.040387,0.072304,0.233541
2,1,3.796029,-0.220110,0.052470,-0.006594,-0.711016,-0.148393,2.766398,0.709664,0.009275,0.273572,0.169401,0.030970,0.247343,0.278714,0.151098
3,3,1.417470,0.951141,1.584397,-0.937251,0.432877,-1.439677,-0.090420,0.522325,0.322685,0.372968,0.098640,0.180842,0.307548,0.040002,0.059175
4,4,1.630352,0.019401,-0.362564,0.066324,-0.650583,0.240731,-1.512800,0.389906,-0.923198,0.021148,0.021058,0.361784,0.550461,0.045550,0.073713


In [27]:
cd_test = CausalityDataset(
    data=test,
    treatment='treatment',
    outcomes=['y_factual'],
    effect_modifiers = features,
    propensity_modifiers = propensities
)

In [28]:
cd_test.preprocess_dataset()

In [29]:
preds_test = ct2.effect(cd_test.data)

- Here we want to get results from the predictions
- We do naive argmax, but it is also recommended to use Thompson Sampling

In [30]:
treatments_test = np.argmax(preds_test, axis=1) + 1

In [31]:
test['predicted_treatment'] = treatments_test
use_df = test

In [32]:
# computing mean ERUPT over 10 bootstrapped samples
from tqdm import tqdm
scores_list = []

for i in tqdm(range(10)):

    bootstrap_df = use_df.sample(frac=1, replace=True)
    propensities = bootstrap_df['propensity_selected_group']
    actual_treatment = bootstrap_df['treatment']
    outcome = bootstrap_df['y_factual']

    # define the random assignment policy
    random_policy = bootstrap_df['predicted_treatment']

    # define a propensity model that will simply return the propensities when calling predict_proba
    propensity_model = DummyPropensity(p=propensities, treatment=actual_treatment)

    # obtain ERUPT under random policy
    e = ERUPT(treatment_name='treatment', propensity_model=propensity_model)
    scores_list.append(e.score(df=use_df,outcome=outcome,policy=random_policy))

erupt_mean = np.mean(scores_list)
erupt_sd = np.std(scores_list)

100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 528.26it/s]


In [33]:
erupt_mean

In [34]:
test['y_factual'].mean()